In [25]:
from pyspark.sql import SparkSession
import re

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL shud") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
    
initList = [("", "")]

df = spark.createDataFrame(initList, schema=["url", "parsed"])
df = df.union(spark.createDataFrame([("https://amazon.com/xx", "false")]))
df = df.union(spark.createDataFrame([("https://patate.com/xx", "true")]))
df = df.dropDuplicates()
#f.show()
for row in df.rdd.collect():
    print(row[0])
    if 'amazon' in row[0] :
        print("Réussi")    


https://amazon.com/xx
Réussi
https://patate.com/xx
